In [2]:
from geopy.distance import great_circle
from math import sin, cos, sqrt, atan2, radians
import pandas as pd
import numpy as np

In [3]:
# 중심 좌표로부터 특정 반경 안에 있는 지하철역 구하기 위한 클래스 정의

class CountNearThings:
  def __init__(self,df,lat,lon,dist=1):
    """
        df: 데이터 프레임
        lat: 중심 위도
        lon: 중심 경도
        dist: 기준 거리(km)"""
    self.df = df
    self.lat = lat
    self.lon = lon
    self.dist = dist

  def filter_by_rectangle(self):
      """
      사각 범위 내 데이터 필터링
      """
      lat_min = self.lat - 0.01 * self.dist
      lat_max = self.lat + 0.01 * self.dist

      lon_min = self.lon - 0.015 * self.dist
      lon_max = self.lon + 0.015 * self.dist

      self.points = [[lat_min, lon_min], [lat_max, lon_max]]

      result = self.df.loc[
          (self.df['lat'] > lat_min) &
          (self.df['lat'] < lat_max) &
          (self.df['lon'] > lon_min) &
          (self.df['lon'] < lon_max)
      ]
      result.index = range(len(result))

      return result

  def filter_by_radius(self, radius):
    """
    반경 범위 내 데이터 필터링
    """
  # 사각 범위 내 데이터 필터링
    tmp = self.filter_by_rectangle()

    # 기준 좌표 포인트
    center = (self.lat, self.lon)

    result = pd.DataFrame()
    for index, row in tmp.iterrows():
        # 개별 좌표 포인트
        point = (row['lat'], row['lon'])
        d = great_circle(center, point).kilometers
        if d <= radius:
            result = pd.concat([result, tmp.iloc[index, :].to_frame().T])

    result.index = range(len(result))

    return result

In [5]:
# 지하철역 파일 가져오기, 중복 지하철역 제거

df = pd.read_csv('subway_location.csv',encoding='cp949')
df = df.rename(columns={'경도': 'lon', '위도': 'lat'})
df = df.drop_duplicates(['역명'])

In [6]:
# 위도, 경도 기반 직선 거리 계산 함수 정의

def mydistance(a1,b1,a2,b2):

  R = 6373.0

  x1 = radians(a1)
  y1 = radians(b1)
  x2 = radians(a2)
  y2 = radians(b2)
  dlon = x2 - x1
  dlat = y2 - y1
  a = sin(dlat / 2)**2 + cos(y1) * cos(y2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  return R * c

In [7]:
# 중심 좌표와 지하철역 거리를 기반으로 가중치 부여 후 합계를 계산해주는 함수 정의

def weight_sum(distance_list):

  weight_list = []

  for k in distance_list:

    if k <= 0.4:
      weight_list.append(3.0)

    elif k > 0.4 and k <= 0.8:
     weight_list.append(2.5)

    elif k >0.8 and k <= 1.2:
      weight_list.append(2.0)

    elif 1.2 > k and k <= 1.6:
     weight_list.append(1.5)

    else:
     weight_list.append(1.0)

  return sum(weight_list)

In [8]:
# 예시 코드

lat = [37.51429, 37.562832, 37.638951]
lon = [127.0621, 127.007815, 127.024745]

station = []

for i in range(len(lat)):

  cnt = CountNearThings(df,lat[i], lon[i], 2)

  result_radius = cnt.filter_by_radius(2)

  distance_list = []

  for j in range(len(result_radius)):
    distance_list.append(mydistance(lat[i],
                                    lon[i],
                                    result_radius.iloc[j][result_radius.columns.get_loc('lat')],
                                    result_radius.iloc[j][result_radius.columns.get_loc('lon')]))

  station.append(weight_sum(distance_list))

print(station)

[9.0, 30.0, 6.0]
